# Import

In [11]:
!pip install "timesfm[torch]" torch --upgrade



  Using cached timesfm-1.3.0-py3-none-any.whl.metadata (15 kB)
Using cached timesfm-1.3.0-py3-none-any.whl (55 kB)


In [30]:
# %% [1] Imports & Setup
import os, re, glob, random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge  # 보정용 (MultiOutput)
from sklearn.multioutput import MultiOutputRegressor

import torch
import timesfm  # 설치됨 가정: pip install "timesfm[torch]"

def set_seed(seed: int = 42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed); torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

LOOKBACK, PREDICT = 28, 7
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FREQ_FOR_DAILY = 0  # TimesFM freq index (일 단위)

TRAIN_PATH = "/Users/jeong-kyu/Documents/LG_Aimers_7기/open/train/train.csv"
TEST_GLOB = "/Users/jeong-kyu/Documents/LG_Aimers_7기/open/test/TEST_*.csv"
SAMPLE_SUB_PATH = "/Users/jeong-kyu/Documents/LG_Aimers_7기/open/sample_submission.csv"
OUT_PATH = "/Users/jeong-kyu/Documents/LG_Aimers_7기/baseline_submission_timesfm_ft_like.csv"

print(f"[INFO] Device: {DEVICE}")


[INFO] Device: cpu


In [31]:
# %% [2] Load train
print(f"[INFO] Load train: {TRAIN_PATH}")
train = pd.read_csv(TRAIN_PATH)
train.head(5)


[INFO] Load train: /Users/jeong-kyu/Documents/LG_Aimers_7기/open/train/train.csv


,영업일자,영업장명_메뉴명,매출수량
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0


In [32]:
# %% [3] Load TimesFM pretrained
HF_REPO = "google/timesfm-1.0-200m-pytorch"  # 필요시 2.0-500m로 교체 가능

tfm_hparams = timesfm.TimesFmHparams(
    backend="gpu" if torch.cuda.is_available() else "cpu",
    per_core_batch_size=32,
    horizon_len=max(128, PREDICT),
    context_len=512,  # v2.0이면 2048도 가능
)

tfm = timesfm.TimesFm(
    hparams=tfm_hparams,
    checkpoint=timesfm.TimesFmCheckpoint(huggingface_repo_id=HF_REPO),
)
print(f"[INFO] Loaded HF repo: {HF_REPO}")


Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 55676.60it/s]


[INFO] Loaded HF repo: google/timesfm-1.0-200m-pytorch


In [33]:
# %% [4] Prepare scalers & last sequences
def prepare_timesfm(train_df: pd.DataFrame) -> dict:
    prepared = {}
    for store_menu, group in tqdm(train_df.groupby(["영업장명_메뉴명"]), desc="Preparing (scalers/last_seq)"):
        g = group.sort_values("영업일자").copy()
        if len(g) < LOOKBACK:
            continue
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(g[["매출수량"]].values.astype(float))
        last_seq = scaled[-LOOKBACK:]  # (28,1)
        prepared[store_menu] = {"scaler": scaler, "last_sequence": last_seq}
    print(f"[INFO] Prepared series: {len(prepared)}")
    return prepared

prepared_cache = prepare_timesfm(train)


Preparing (scalers/last_seq): 100%|██████████| 193/193 [00:00<00:00, 1289.05it/s]

[INFO] Prepared series: 193


In [34]:
# %% [5] Build calibration (fine-tuning-like) dataset
def build_calib_pairs(train_df: pd.DataFrame,
                      use_per_series_model: bool = False):
    """
    Returns:
      if use_per_series_model:
        dict[series_key] = { 'X': np.ndarray[n_samples, PREDICT], 'y': np.ndarray[n_samples, PREDICT] }
      else:
        global_X, global_y
    """
    if use_per_series_model:
        out = {}

    global_X, global_y = [], []

    for key, group in tqdm(train_df.groupby(["영업장명_메뉴명"]), desc="Building calibration pairs"):
        g = group.sort_values("영업일자").copy()
        if len(g) < LOOKBACK + PREDICT:
            continue

        # 시리즈별 스케일러 (예측/학습 일관성)
        scaler = MinMaxScaler()
        vals = scaler.fit_transform(g[["매출수량"]].values.astype(float)).reshape(-1)

        X_list, y_list = [], []
        # 슬라이딩: lookback을 입력, 이후 predict 길이를 타깃
        for i in range(len(vals) - LOOKBACK - PREDICT + 1):
            x_seq = vals[i:i+LOOKBACK]                     # (28,)
            y_seq = vals[i+LOOKBACK:i+LOOKBACK+PREDICT]    # (7,)

            # TimesFM로 예측(포인트)
            pred, _ = tfm.forecast([x_seq], freq=[FREQ_FOR_DAILY])
            pred = np.asarray(pred[0])[:PREDICT]           # (7,)

            X_list.append(pred)
            y_list.append(y_seq)

        if not X_list:
            continue

        X_arr = np.stack(X_list, axis=0)
        y_arr = np.stack(y_list, axis=0)

        if use_per_series_model:
            out[key] = {"X": X_arr, "y": y_arr}
        else:
            global_X.append(X_arr)
            global_y.append(y_arr)

    if use_per_series_model:
        return out
    else:
        if not global_X:
            raise RuntimeError("No calibration pairs built. Check train length.")
        return np.concatenate(global_X, axis=0), np.concatenate(global_y, axis=0)

# 글로벌 보정기(권장) — 시리즈별로 하고 싶으면 use_per_series_model=True로 바꾸세요.
calib_X, calib_y = build_calib_pairs(train, use_per_series_model=False)
calib_X.shape, calib_y.shape


Building calibration pairs:   8%|▊         | 15/193 [38:13<7:33:31, 152.87s/it]


KeyboardInterrupt: 

In [ ]:
# %% [6] Train calibration model (multi-output Ridge)
calibrator = MultiOutputRegressor(Ridge(alpha=1.0, fit_intercept=True, random_state=42))
calibrator.fit(calib_X, calib_y)
print("[INFO] Calibrator trained (global)")


In [ ]:
# %% [7] Predict function with calibration
def predict_timesfm_with_calib(test_df: pd.DataFrame,
                               prepared: dict,
                               test_prefix: str,
                               use_per_series_model: bool = False,
                               per_series_calibrators: dict | None = None,
                               global_calibrator: MultiOutputRegressor | None = None):
    results = []
    skipped = 0

    for key, g in test_df.groupby(["영업장명_메뉴명"]):
        if key not in prepared:
            skipped += 1
            continue

        scaler = prepared[key]["scaler"]
        last_seq_train = prepared[key]["last_sequence"].reshape(-1)  # (28,)

        g = g.sort_values("영업일자")
        recent = g["매출수량"].values.astype(float)[-LOOKBACK:]

        if len(recent) < LOOKBACK:
            x_scaled = last_seq_train
        else:
            x_scaled = scaler.transform(recent.reshape(-1,1)).reshape(-1)

        # TimesFM 포인트 예측
        pred, _ = tfm.forecast([x_scaled], freq=[FREQ_FOR_DAILY])
        pred = np.asarray(pred[0])[:PREDICT].reshape(1, -1)  # (1,7)

        # 보정 적용
        if use_per_series_model and per_series_calibrators is not None and key in per_series_calibrators:
            pred_corr = per_series_calibrators[key].predict(pred)[0]
        else:
            pred_corr = global_calibrator.predict(pred)[0]

        # 역스케일 + 음수 컷
        restored = []
        for v in pred_corr:
            restored_val = scaler.inverse_transform(np.array([[v]]) )[0, 0]
            restored.append(max(restored_val, 0.0))

        dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]
        for d, val in zip(dates, restored):
            results.append({"영업일자": d, "영업장명_메뉴명": key, "매출수량": val})

    if skipped:
        print(f"[WARN] train에 없던 키 {skipped}개 스킵됨")
    return pd.DataFrame(results)


In [ ]:
# %% [8] Predict over all test files
all_preds = []
test_files = sorted(glob.glob(TEST_GLOB))
print(f"[INFO] Found test files: {len(test_files)}")

for path in test_files:
    if os.path.isdir(path):
        continue
    test_df = pd.read_csv(path)
    filename = os.path.basename(path)
    m = re.search(r"(TEST_\d+)", filename)
    if not m:
        print(f"[WARN] prefix not found: {filename}")
        continue
    test_prefix = m.group(1)

    pred_df = predict_timesfm_with_calib(
        test_df,
        prepared=prepared_cache,
        test_prefix=test_prefix,
        use_per_series_model=False,
        per_series_calibrators=None,
        global_calibrator=calibrator,
    )
    if not pred_df.empty:
        all_preds.append(pred_df)

if not all_preds:
    raise RuntimeError("[ERROR] 예측 결과가 비었습니다.")

full_pred_df = pd.concat(all_preds, ignore_index=True)
full_pred_df.head()


In [ ]:
# %% [9] Convert to submission & save
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    final_df = sample_submission.copy()
    pred_df = pred_df.copy()
    pred_df['영업장명_메뉴명'] = pred_df['영업장명_메뉴명'].apply(
        lambda x: (x[0] if isinstance(x, (list, tuple)) else x)
    ).astype(str).str.strip()
    pred_df['영업일자'] = pred_df['영업일자'].astype(str).str.strip()

    pred_agg = (pred_df
                .groupby(['영업일자', '영업장명_메뉴명'], as_index=True)['매출수량']
                .sum())
    pred_wide = pred_agg.unstack(fill_value=0)

    id_col = final_df.columns[0]
    final_df[id_col] = final_df[id_col].astype(str)
    dates = final_df[id_col].tolist()

    sample_cols = list(final_df.columns[1:])
    sample_col_norm_map = {col: str(col).strip() for col in sample_cols}

    pred_wide = pred_wide.reindex(dates)
    fill_vals = {}
    for raw_col in sample_cols:
        norm_col = sample_col_norm_map[raw_col]
        if (pred_wide is not None) and (norm_col in pred_wide.columns):
            fill_vals[raw_col] = pred_wide[norm_col].to_numpy()
        else:
            fill_vals[raw_col] = np.zeros(len(dates), dtype=float)

    for raw_col, arr in fill_vals.items():
        final_df[raw_col] = arr

    return final_df

sample_submission = pd.read_csv(SAMPLE_SUB_PATH)
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print(f"[INFO] Saved: {OUT_PATH}")
